In [1]:
import numpy as np
from numpy.linalg import norm
from time import time
from help_functions import diag_matrice_croissant

In [2]:
def kernel_cost(gamma, data, reg):

    KX2 = data['KX2']
    KY2 = data['KY2']
    KX3 = data['KX3']
    KY3 = data['KY3']

    tmp1 = np.mean(KX3) + np.mean(KY3)
    tmp2 = (np.mean(KX2, axis=0) + np.mean(KY2, axis=0)) @ gamma
    c = (tmp1 - tmp2) / (2 * reg)

    return c


In [3]:
def gradient(gamma, X, Phi, Q, z, reg1, reg2):
    m = len(z)
    H = Phi.T @ X @ Phi
    g_gamma = (Q @ gamma - z) / (2 * reg2) - np.diag(H)
    g_X = Phi @ np.diag(gamma) @ Phi.T + reg1 * np.eye(m)
    return g_gamma, g_X

In [ ]:
def residue(gamma, X, Phi, Q, z, reg1, reg2):
    r_gamma, g_X = gradient(gamma, X, Phi, Q, z, reg1, reg2)
    X_new = X - g_X
    D, V = np.linalg.eig(X_new)
    X_new = V @ np.maximum(D, 0) @ V.T
    r_X = X - X_new
    return r_gamma, r_X

In [ ]:
def SSN_main(r_gamma, r_X, gamma, X, mu, Q, Phi, reg1, reg2):
  '''
  Cette partie correspond
  '''
  m = len(gamma)

  # the first step
  Z = X - (Phi @ np.diag(gamma) @ Phi.T + reg1 * np.eye(m)) #On commence par calculer la la matrice Z_{k}:= X_{k}-(Phi^{*}(gamma_{k})+lambda_{1}I) Pour simplifier les calculs on écrit Phi^{*} sous sa forme matricielle plutot.
  Sigma, P = diag_matrice_croissant(Z)# On diagonalise Z et on réarrange la décomposition de façon à ce que les valeurs propres soient triées par ordre croissant
  sigma=Sigma.diagonal() #Une liste avec les valeurs propres de Z dans l'ordre décroissant.
  alpha = sigma[sigma > 0]#On crée une liste des valeurs propres de Z strictement positifs
  beta = sigma[sigma <= 0] #On crée une liste des valeurs propres de Z strictement positifs

  #Construction de la matrice Omega
  
  Omega = np.zeros((m, m)) #On va désormais créer la matrice Omega. Pour cela, on prend une matrice nulle qu'on remplit comme décrit dans l'article
  Omega[:len(alpha), :len(alpha)] = np.ones((len(alpha), len(alpha))) #D'abord, on va créer la partie supérieure gauche remplie de 1 là où les valeurs propres sont positives.
  eta = np.array([[a / (b-a) for b in beta] for a in alpha])# On construit la matrice eta comme décrit dans l'article.
  Omega[:len(alpha), len(alpha):len(alpha)+len(beta)] = eta #On remplit la partie supérieure droite de Omega avec eta.
  Omega[len(alpha):, :len(alpha)] = eta.T #On remplit la partie inférieure gauche par la tranposée de eta ce qui finit de construire Omega.
  
  #Une fois on a obtenu Omega on peut construire Gamma et L
  
  Gamma=np.diag(np.diag(Omega)) #On crée la matrice Gamma qui est la diagonale de Omega.
  L=np.diag(np.diag(Gamma)/(mu-np.diag(Gamma)+1)) #On crée la matrice L comme décrit dans le lemme 4.3. 
  
  #On va maintenant construire la matrice Psi à partir de la matrice Omega.
  
  Psi=np.zeros((m, m))# On va créer la matrice Psi pour la construction de l'opérateur T.
  Psi[:len(alpha), :len(alpha)] = (1/mu)*np.ones((len(alpha), len(alpha)))

  T = r_X + P @ (L * (P.T @ r_X @ P)) @ P.T
  H = Phi.T @ T @ Phi
  d_gamma = -r_gamma - np.diag(H) / (1 + mu)
  d_X = -r_X

  # the second step (CG)
  y = d_gamma
  K = P.T @ Phi
  H = K.T @ (L * (K @ np.diag(y) @ K.T)) @ K
  r = d_gamma - ((0.5 / reg2) * Q @ y + mu * y + np.diag(H))
  p = r
  rr = r @ r.T
  for i in range(min(m // 5, 50)):
        H = K.T @ (L * (K @ np.diag(p) @ K.T)) @ K
        Ap = (0.5 / reg2) * Q @ p + mu * p + np.diag(H)
        ss1 = rr / (p @ Ap.T)
        y = y + ss1 * p
        r = r - ss1 * Ap
        if np.linalg.norm(r) < 1e-6:
            break
        ss2 = r @ r.T / rr
        p = r + ss2 * p
        H = K.T @ (L * (K @ np.diag(y) @ K.T)) @ K
        r = d_gamma - ((0.5 / reg2) * Q @ y + mu * y + np.diag(H))
        rr = r @ r.T
  d_gamma = y
  d_X = (d_X + P @ (L * (P.T @ d_X @ P)) @ P.T) / (1 + mu)

    # the third step
  d_X = d_X - (P @ (L * (K @ np.diag(d_gamma) @ K.T)) @ P.T)

  return d_gamma, d_X


In [ ]:


def SSN(data, reg1, reg2, verbose=False):
    # input data
    M = data['M']
    Phi = data['Phi']
    KX1 = data['KX1']
    KY1 = data['KY1']
    KX2 = data['KX2']
    KY2 = data['KY2']

    # initialization
    m = len(M)
    Q = KX1 + KY1
    z = np.mean(KX2, axis=0) + np.mean(KY2, axis=0) - 2 * reg2 * M
    nIter = 300

    gamma = np.ones(m) / m
    X = np.ones((m, m)) / (m * m)
    kappa = 1.0
    r_gamma, r_X = residue(gamma, X, Phi, Q, z, reg1, reg2)
    mu = norm(r_gamma) + norm(r_X, 'fro')
    res_time = [0]
    res_norm = [mu]

    if verbose:
        print('\n-------------- SSNEG ---------------')
        print('iter |  cost  |  residue  |  time')

    tstart = time()

    # main loop
    for iter in range(1, nIter + 1):
        # compute the residue function
        mu = norm(r_gamma) + norm(r_X, 'fro')

        # compute SSN step
        d_gamma, d_X = SSN_main(r_gamma, r_X, gamma, X, (m / 5) * kappa * mu, Q, Phi, reg1, reg2)

        # compute the next iterate
        gamma = gamma + d_gamma
        X = X + d_X

        # update the parameter kappa.
        r_gamma, r_X = residue(gamma, X, Phi, Q, z, reg1, reg2)
        rho = -(np.dot(r_gamma, d_gamma) + np.trace(np.dot(r_X.T, d_X))) / (norm(d_gamma) ** 2 + norm(d_X, 'fro') ** 2)
        if rho >= 1:
            kappa = max(0.5 * kappa, 1e-16)
        elif rho >= 1e-6:
            kappa = 1.2 * kappa
        else:
            kappa = 25 * kappa

        if mu < 1e-8:  # 5e-3
            c = kernel_cost(gamma, data, reg2)
            t = time() - tstart
            res_time.append(t)
            res_norm.append(mu)
            if verbose:
                print('%5.0f|%3.2e|%3.2e|%3.2e' % (iter, c, mu, t))
            break

        if iter % 30 == 0:
            c = kernel_cost(gamma, data, reg2)
            t = time() - tstart
            res_time.append(t)
            res_norm.append(mu)
            if verbose:
                print('%5.0f|%3.2e|%3.2e|%3.2e' % (iter, c, mu, t))

        c = kernel_cost(gamma, data, reg2)
        t = time() - tstart

    return gamma, c, t, res_time, res_norm


In [61]:
X=np.ones((4,4))

In [62]:
diag=np.diag(X)

In [64]:
A = np.diag(np.diag(X))

In [66]:
L = np.diag(np.diag(A)/(np.diag(A)+1))

In [67]:
L

array([[0.5, 0. , 0. , 0. ],
       [0. , 0.5, 0. , 0. ],
       [0. , 0. , 0.5, 0. ],
       [0. , 0. , 0. , 0.5]])

In [6]:
Sigma,P=diag_matrice_croissant(X)

In [27]:
sigma=Sigma.diagonal()

In [39]:
alpha = sigma[sigma > 0]#On crée une liste des valeurs propres de Z strictement positifs
beta = sigma[sigma <= 0]